# Getting started

This notebook provides an introduction to the tools that will be used for the _hands-on_ sessions of the MOOC "Path Integrals in Atomistic Modeling". Please follow the instructions to make sure that you have all the necessary components in place. 

## Installation

The notebooks associated with this course rely on some basic Python packages. If executing the following cell returns any errors, you should install the corresponding packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pimdmooc
pimdmooc.add_ipi_paths()

You should also have [i-PI](https://ipi-code.org) installed, and accessible from the path. This might require some more effort: if everything is configured correctly the following cell should return a prompt, and a message stating that 

```Simulation has already run for total_steps, will not even start. Modify total_steps or step counter to continue.```

In [ ]:
!i-pi 0-getting_started/do_nothing.xml

... and the following cell should generate a help string explaining the syntax for running `i-pi-driver`

In [ ]:
!i-pi-driver

If you get an error stating that the command has not been found, please open a terminal and follow these instructions.

_NB: this will install i-PI in your home folder, which will make these notebooks work out-of-the-box. if you know what you are doing, you can personalize the installation as long as `i-pi` and `i-pi-driver` are accessible from the path_

1. clone the i-PI repository

``` 
$ git clone https://github.com/i-pi/i-pi.git 
```

2. add the i-pi folder to the default path

```
$ echo ". ~/i-pi/env.sh" >> .bashrc
```

3. compile the driver files 

```
$ cd i-pi/drivers/f90/ & make
```

4. try to execute again the cells above

## Running i-PI

i-PI functions following a client-server protocol, in which i-PI acts as the server, executing advanced MD simulations, while the evaluation of energies and forces is delegated to an external code, that acts as the client

![a scheme of the i-PI client-server model](figures/ipi-scheme.png)

Thus, to run i-PI you need to launch (at least) two processes: `i-pi` and a driver -- here we use a minimalistic FORTRAN tool that can compute energy and forces according to a number of simple potential energy models. 

1. open *two* terminals
2. in the first terminal launch `i-pi`

```
$ cd pimd-mooc/0-getting_started
$ i-pi input.xml
```

3. in the second terminal, launch the driver

```
$ i-pi-driver -u -h driver -m pswater -v
```

Both programs run in verbose mode, so you can see the communication logs between the two. 

```
 @SOCKET:   Client asked for connection from . Now hand-shaking.
 @SOCKET:   Handshaking was successful. Added to the client list.
 @SOCKET: 21/09/17-15:49:35 Assigning [ none] request id    0 to client with last-id None (  0/  1 : )
 @SOCKET: 21/09/17-15:49:35 Assigning [match] request id    0 to client with last-id    0 (  0/  1 : )
 # Average timings at MD step       0. t/step: 4.08595e-02
 @SOCKET: 21/09/17-15:49:35 Assigning [match] request id    0 to client with last-id    0 (  0/  1 : )
 # Average timings at MD step       1. t/step: 3.96178e-02
 @SOCKET: 21/09/17-15:49:35 Assigning [match] request id    0 to client with last-id    0 (  0/  1 : )
```

and 

```
  Message from server: STATUS
  Message from server: POSDATA
  Message from server: STATUS
  Message from server: GETFORCE
  Message from server: STATUS
  Message from server: STATUS
  Message from server: POSDATA
  Message from server: STATUS
  Message from server: GETFORCE
  Message from server: STATUS
```

In [ ]:
! cd 0-getting_started/ && i-pi input.xml & 